In [33]:
from pymongo import MongoClient
import json

In [34]:
c=MongoClient()
oa=c["openalexco"]["works"]
kh=c["kahi_dev"]["works"]
khp=c["kahi_dev"]["person"]

In [35]:
pipeline = [
    {"$unwind": "$authors"},

    {
        "$addFields": {
            "authors.affiliations": {
                "$filter": {
                    "input": "$authors.affiliations",
                    "as": "aff",
                    "cond": {
                        "$in": [
                            {"source": "ror", "type": "Education"},
                            "$$aff.types"
                        ]
                    }
                }
            }
        }
    },

    {
        "$match": {
            "$expr": {"$gt": [{"$size": "$authors.affiliations"}, 1]}
        }
    },

    {"$replaceRoot": {"newRoot": "$$ROOT"}},
    {
        "$project": {
            "_id": 1,
            "authors": 1,
            "titles": 1,
            "external_ids": {
                "$filter": {
                    "input": "$external_ids",
                    "as": "ext",
                    "cond": {"$eq": ["$$ext.source", "openalex"]}
                }
            }
        }
    }

]

ws=kh.aggregate(pipeline)

In [36]:
ws=list(ws)

In [37]:
len(ws)

377010

In [38]:
ws[0]

{'_id': ObjectId('68ade991a6d1f814a32696ac'),
 'titles': [{'title': 'Política pública intersectorial a partir de la planificación cultural. Economía, ciencia y tecnología en la planeación cultural',
   'lang': 'es',
   'source': 'openalex'},
  {'title': "Polı́tica pública intersectorial a partir de la planificación cultural. Econom\\á, ciencia y tecnolog\\' ́en la planeación cultural.",
   'lang': 'es',
   'source': 'scholar'},
  {'title': 'Política pública intersectorial a partir de la planificación cultural. Economía, ciencia y tecnología en la planeación cultural',
   'lang': 'es',
   'source': 'minciencias'},
  {'title': 'Política pública intersectorial a partir de la planificación cultural. Economía, ciencia y tecnología en la planeación cultural.',
   'lang': 'es',
   'source': 'dspace'},
  {'title': 'Intersectoral public policy from cultural planning. Economy, science and technology in the cultural planning.',
   'lang': 'en',
   'source': 'dspace'}],
 'authors': {'id': '0000782

In [39]:
def get_oa_author_id(kid):
    pipe=[{
        "$match":{"_id":kid}
    },
        {
        "$project": {
            "external_ids": {
                "$filter": {
                    "input": "$external_ids",
                    "as": "ext",
                    "cond": {"$eq": ["$$ext.source", "openalex"]}
                }
            }
        }
    }
    ]
    _id = list(khp.aggregate(pipe))
    return _id[0]["external_ids"][0]["id"] if _id and _id[0]["external_ids"] else None


In [40]:
ws[0]["authors"]["id"]

'0000782025'

In [50]:
found = []
for w in ws[0:1000]:
    _id = get_oa_author_id(w["authors"]["id"])
    fw=oa.find_one({"id":w["external_ids"][0]["id"]},{"authorships":1})
    for a in fw["authorships"]:
        if a["author"]["id"] == _id and len(a["institutions"]) != len(w["authors"]["affiliations"]):
            print(len(a["institutions"]),len(w["authors"]["affiliations"]))
            #print(w["external_ids"][0]["id"])
            found.append(w)


1 3
1 2
1 2
1 2
4 3
1 2
1 3
1 2
1 2
1 2
1 2
0 2
1 2
1 2
1 2
1 3
1 2
1 2
1 2
1 2
1 2
1 2
7 6
1 2
1 2
4 2
3 2
1 2
1 2
3 2
3 2
3 2
3 2
1 2
1 4
5 3
5 3
5 3
1 2
1 2
1 2
1 2
3 2
1 4
6 5
1 2
1 2
1 2
1 2
0 2
4 3
1 2
2 3
3 2
3 2
3 2
3 2
1 2
1 3
1 2
1 2
2 4
1 2
1 2
1 2
1 2
3 2
1 2
1 2
4 2
1 2
6 4
1 2
4 2
3 2
1 2
1 2
1 2
1 2
1 2
3 2
3 2
4 3
1 2
4 2
1 2
1 3
0 2
1 2
1 2
4 3
1 3
1 2
1 2
1 2
3 2
4 2
3 2
1 2
1 2
3 2
0 2
1 2
1 3
1 4
1 3
1 2
1 3
3 2
1 2
1 5
1 2
1 2
8 3
8 3
8 3
8 3
8 3
8 3
8 3
8 3
8 3
8 3
1 2
1 2
1 2
1 2
2 4
1 4
3 2
4 2
1 2
1 2
3 2
4 3
1 4
1 2
1 2
1 3
1 2
1 2
1 2
5 4
0 2
1 2
1 2
3 2
3 2
3 2
3 2
3 2
1 2
1 3
1 2
3 2
4 3
1 2
4 2
3 2
1 2
4 2
3 2
3 2
1 2
1 3
1 2
1 2
1 2
3 2
3 2
3 2
3 2
1 2
1 2
0 2
1 2
1 4
1 2
0 3
1 2
1 2
3 2
1 2
1 2
1 2
1 2
3 2
4 2
1 2
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3
5 3


In [45]:
len(found)

496

In [46]:
for w in found:
    w["_id"]=str(w["_id"])

In [49]:
f=open("comet.json","w")
f.write(json.dumps(found))
f.close()